In [9]:
import bigframes
import bigframes.streaming
import bigframes.pandas as bpd
import google.cloud.bigquery as bigquery
print(bigframes.__version__)

1.6.0


In [10]:
bpd.close_session()
PROJECT_ID = "henryjsolberg-prod"  # @param {type:"string"}
REGION = "US"  # @param {type: "string"}
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = REGION
sql = """SELECT
body_mass_g as string, island as rowkey
FROM birds.penguins"""

In [17]:
query_job = bigframes.streaming.write_stream_bigtable(
    sql,
    "streaming-testing-instance",
    "table-testing",
)

In [12]:
query_job.done()

False

In [18]:
query_job.cancel()

True

In [19]:
query_job.done()

True

In [15]:
client = bigframes.get_global_session().bqclient
sql = """EXPORT DATA
 OPTIONS (
 format = 'CLOUD_BIGTABLE',
 truncate = TRUE,
 overwrite = TRUE,
 uri = "https://bigtable.googleapis.com/projects/henryjsolberg-prod/instances/streaming-testing-instance/tables/table-testing"
 )
AS (
SELECT
body_mass_g as string, island as rowkey
FROM birds.penguins);"""
job_config = bigquery.job.QueryJobConfig()
job_config = job_config.from_api_repr({"query": {"continuous": True}})
job_config.to_api_repr()

{'query': {'continuous': True}}

In [16]:
_, query_job = bigframes.session._io.bigquery.start_query_with_client(
    client, #bq_client: bigquery.Client,
    sql, #sql: str,
    job_config
)

Requested cancellation for Query job c331db5d-cde5-42ef-9745-2a4d2f0b3d00 in location US...


KeyboardInterrupt: 

In [ ]:
client.project
client.location
#query_job.done()

'US'